In [ ]:
import os
import pandas as pd

from pathlib import Path


In [ ]:
root = Path(os.path.abspath('')).parent.resolve()

df_train = pd.read_csv(root / "data" / "train.csv")
df_test = pd.read_csv(root / "data" / "test.csv")
df_geography = 